In [ ]:
import os
from collections import defaultdict
from vlmeval.smp import ls, load, dump, download_file, encode_image_file_to_base64, md5
import pandas as pd

In [ ]:
download_file('https://ai2-public-datasets.s3.amazonaws.com/diagrams/ai2d-all.zip')
os.system('unzip -o ai2d-all.zip')

images = ls('ai2d/images/')
questions = ls('ai2d/questions/')
cates = load('ai2d/categories.json')

In [ ]:
data_all = defaultdict(list)
for qfile in questions:
    data = load(qfile)
    image_pth = qfile.replace('questions', 'images').replace('.json', '')
    cate = cates[image_pth.split('/')[-1]]
    for q, qmeta in data['questions'].items():
        assert '.png-' in qmeta['questionId']
        main, sub = qmeta['questionId'].split('.png-')
        idx = int(main) * 100 + int(sub)
        data_all['index'].append(idx)
        data_all['question'].append(q)
        data_all['image_path'].append(image_pth)
        answers = qmeta['answerTexts']
        correct = qmeta['correctAnswer']
        assert len(answers) == 4
        for c, a in zip('ABCD', answers):
            data_all[c].append(a)
        data_all['answer'].append('ABCD'[qmeta['correctAnswer']])
        data_all['category'].append(cate)
data_all = pd.DataFrame(data_all)

In [ ]:
data = data_all
print(data.keys())

In [ ]:
images = []
image_seen = {}
for idx, pth in zip(data['index'], data['image_path']):
    if pth in image_seen:
        images.append(image_seen[pth])
    else:
        image_seen[pth] = idx
        images.append(encode_image_file_to_base64(pth))

data['image'] = images
dump(data, 'AI2D.tsv')
print(md5('AI2D.tsv'))